In [1]:
import os 
import json 

import boto3
import plotly.express as px
import pandas as pd

from mirrorverse.utils import read_data_w_cache

os.environ['HAVEN_DATABASE'] = 'haven'
os.environ['AWS_PROFILE'] = 'admin'

In [106]:
s3_client = boto3.client('s3')
bucket = 'mimic-log-odds-models'
prefix = 'movement-model-m3-a3-v12'

response = s3_client.list_objects_v2(Bucket=bucket, Prefix=prefix)

configs = {}
if 'Contents' in response:
    for obj in response['Contents']:
        if obj['Key'].endswith('config.json'):
            config = json.loads(s3_client.get_object(Bucket=bucket, Key=obj['Key'])['Body'].read().decode('utf-8'))
            if 'model' in config:
                configs[config['run_id']] = config['model']
else:
    print("No objects found.")

In [31]:
results = (
    read_data_w_cache('select * from movement_model_experiment_m3_a4_v1')
    .sort_values(['run_id', 'epoch'], ascending=True)
)
results.head()

,loss,val_loss,epoch,experiment_name,run_id
3100,0.259766,0.163177,1,movement-model-m3-a4-v1,01399f21363800ae1b028d00866bc0902ab960b895166e...
3101,0.185548,0.162778,2,movement-model-m3-a4-v1,01399f21363800ae1b028d00866bc0902ab960b895166e...
3102,0.184732,0.162998,3,movement-model-m3-a4-v1,01399f21363800ae1b028d00866bc0902ab960b895166e...
3103,0.182802,0.163240,4,movement-model-m3-a4-v1,01399f21363800ae1b028d00866bc0902ab960b895166e...
3104,0.181459,0.162140,5,movement-model-m3-a4-v1,01399f21363800ae1b028d00866bc0902ab960b895166e...


In [32]:
final = results.groupby('run_id')[['val_loss', 'loss', 'epoch']].last().reset_index().sort_values('val_loss', ascending=True)
#final = final[final['epoch'] < 100]
final.head()

,run_id,val_loss,loss,epoch
137,b67145fad126d9b32940aef7aa8636d6d0b2682b2b6d75...,0.156296,0.175063,50
59,47ee6fdc3d339cabd2f21ae9798628c76c261a21a42475...,0.156471,0.178513,50
24,22931f94ddc05e16fd40232ea5ebb26ce96a44251a87c3...,0.156650,0.180240,50
52,3c08c66884094ad49c5df2735da75043dc7e7b8553ebf4...,0.157589,0.193893,50
88,6b9ad41e5e5af8cd829c78dec39801761c1a81766a6dfd...,0.157610,0.177817,50


In [121]:
def count_layers(layers):
    layers = [int(l[1:]) for l in layers]
    return sum(layers)

config_df = pd.DataFrame([
    {
        'run_id': run_id, 
        'epochs': config['epochs'], 
        'batch_size': config['batch_size'],
        'depth': len(config['layers']),
        'size': int(config['layers'][0][1:])*3,
        'dropout': config['layers'][1].startswith('Drop')
    }
    for run_id, config in configs.items()
])
config_df.head()

,run_id,epochs,batch_size,depth,size,dropout
0,00c53a805980b861ffcf407d7243ae55d1bb55fb80ba6f...,125,600,6,24,True
1,0a1eecf7b4bc4131da11ff9ada46802a6cb1aa97773a50...,125,600,6,48,True
2,1042d679d85b1263dff25144f3ac1dd201f741464955ff...,125,700,3,24,False
3,1999611940824648350a19b7530d3bc1da790b94eddd79...,125,500,3,72,False
4,32d49ea9c21928022a422da5bb54470c99d86230e30dc0...,125,600,6,72,True


In [36]:
x = final.sort_values('val_loss', ascending=True)['run_id'].values[0]
x

'b67145fad126d9b32940aef7aa8636d6d0b2682b2b6d753c595e067fb44b3485'

In [13]:
x = config_df[config_df['size'] == 24 * 3]['run_id'].values[5]

NameError: name 'config_df' is not defined

In [140]:
configs[x]

{'batch_size': 600,
 'dropout': 0,
 'epochs': 125,
 'layer_size': 24,
 'layers': ['D24', 'D24', 'D24']}

In [34]:
px.line(results[results['run_id'] == x], x='epoch', y='loss')

In [11]:
px.scatter(final, x='loss', y='val_loss')

,loss,val_loss,epoch,experiment_name,run_id
750,0.371597,0.199504,1,movement-model-m3-a3-v11,00f4a3532039ee385a7fd2b9a10cb0013d8dd71004120b...
751,0.179280,0.178424,2,movement-model-m3-a3-v11,00f4a3532039ee385a7fd2b9a10cb0013d8dd71004120b...
752,0.169080,0.172203,3,movement-model-m3-a3-v11,00f4a3532039ee385a7fd2b9a10cb0013d8dd71004120b...
753,0.164986,0.171374,4,movement-model-m3-a3-v11,00f4a3532039ee385a7fd2b9a10cb0013d8dd71004120b...
754,0.163007,0.170574,5,movement-model-m3-a3-v11,00f4a3532039ee385a7fd2b9a10cb0013d8dd71004120b...
...,...,...,...,...,...
313,0.136698,0.174842,196,movement-model-m3-a3-v11,fcc5f7d6aca164072257fd9ea5cdf3e8fbb6ae1a88c07b...
314,0.136469,0.173970,197,movement-model-m3-a3-v11,fcc5f7d6aca164072257fd9ea5cdf3e8fbb6ae1a88c07b...
315,0.136598,0.174965,198,movement-model-m3-a3-v11,fcc5f7d6aca164072257fd9ea5cdf3e8fbb6ae1a88c07b...
316,0.136474,0.174210,199,movement-model-m3-a3-v11,fcc5f7d6aca164072257fd9ea5cdf3e8fbb6ae1a88c07b...


In [28]:
px.scatter(final, x='loss', y='val_loss')

,run_id,epochs,batch_size,neurons,depth,layer1,layer2
0,00f4a3532039ee385a7fd2b9a10cb0013d8dd71004120b...,150,700,28,3,8,16
1,05a71a1d2dda24d43c6b6d6eef286171ac9da1019713b6...,200,500,16,2,8,8
2,0e78cae42f1524fad7bf3e56d071811d5ffb1155a74e36...,200,600,32,2,16,16
3,258c11d48f8b6ca12420b187b13085f2f730c5198643f4...,250,500,48,3,16,16
4,3a96508d290d51b52fa9c96791de46fb4f3b3fda4ccfbb...,150,500,28,3,8,16


In [16]:
df = final.merge(config_df)
px.scatter(df, x='epochs', y='loss')

In [17]:
px.scatter(df, x='batch_size', y='loss')

In [18]:
px.scatter(df, x='neurons', y='loss')

In [19]:
px.scatter(df, x='depth', y='loss')

In [11]:
px.scatter(df, x='layer1', y='val_loss')

In [12]:
px.scatter(df, x='layer2', y='val_loss')

In [8]:
x = final.sort_values('val_loss', ascending=True)['run_id'].values[0]
x

'b30b94e5e56699af469c56a55912970cde3e72ab51f0635bab2af064c3bc8684'

In [14]:
configs[x]

{'batch_size': 600, 'epochs': 15, 'layers': ['D8', 'D16', 'D8']}

In [15]:
results[results['run_id'] == x]

,loss,val_loss,epoch,experiment_name,run_id
350,0.324010,0.185659,1,movement-model-m3-a3-v5,6f2050e2a211fe3119e3ea28f1f182d49bad87ad0a5e80...
351,0.172655,0.162878,2,movement-model-m3-a3-v5,6f2050e2a211fe3119e3ea28f1f182d49bad87ad0a5e80...
352,0.163672,0.159378,3,movement-model-m3-a3-v5,6f2050e2a211fe3119e3ea28f1f182d49bad87ad0a5e80...
353,0.160549,0.161644,4,movement-model-m3-a3-v5,6f2050e2a211fe3119e3ea28f1f182d49bad87ad0a5e80...
354,0.158714,0.158703,5,movement-model-m3-a3-v5,6f2050e2a211fe3119e3ea28f1f182d49bad87ad0a5e80...
355,0.157374,0.158391,6,movement-model-m3-a3-v5,6f2050e2a211fe3119e3ea28f1f182d49bad87ad0a5e80...
356,0.156245,0.159933,7,movement-model-m3-a3-v5,6f2050e2a211fe3119e3ea28f1f182d49bad87ad0a5e80...
357,0.155520,0.160173,8,movement-model-m3-a3-v5,6f2050e2a211fe3119e3ea28f1f182d49bad87ad0a5e80...
358,0.154903,0.159318,9,movement-model-m3-a3-v5,6f2050e2a211fe3119e3ea28f1f182d49bad87ad0a5e80...
359,0.154140,0.157888,10,movement-model-m3-a3-v5,6f2050e2a211fe3119e3ea28f1f182d49bad87ad0a5e80...
